## 連接ES

In [2]:
import pandas as pd
from elasticsearch import Elasticsearch
import random
import string

es_kwargs = {
    "hosts": "https://localhost:9200",
    "basic_auth": ('elastic', 'pass.123')
}

es = Elasticsearch(**es_kwargs, ca_certs='./ca.crt')
es_info = es.info()
print(f"Connected to cluster named '{es_info['cluster_name']}' (version: {es_info['version']['number']})")


Connected to cluster named 'docker-cluster' (version: 8.15.0)


## 建立索引並插入資料

In [3]:
from eland import pandas_to_eland

index_name = "random_data_index"

if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

def generate_random_data(n):
    data = []
    for i in range(n):
        random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=10))
        data.append({
            "id": i,
            "random_string": random_string,
            "random_number": random.randint(1, 1000)
        })
    return data

num_records = 20000
random_data = generate_random_data(num_records)

df = pd.DataFrame(random_data)

ed = pandas_to_eland(
    pd_df=df,
    es_client=es,
    es_dest_index=index_name,
    es_if_exists="replace",
    es_refresh=True
)



## 驗證資料是否插入成功

In [7]:
print(f"成功插入 {num_records} 筆資料到Index '{index_name}'")
count = es.count(index=index_name)
print(f"Index '{index_name}' 中的documnet數量: {count['count']}")

成功插入 20000 筆資料到Index 'random_data_index'
Index 'random_data_index' 中的documnet數量: 20000


## 驗證eland是否可以讀出超過一萬筆資料

In [8]:
import eland

# 從ES中讀取資料
ed_df = eland.DataFrame(es_client=es, es_index_pattern=index_name)

total_rows = len(ed_df)
print(f"Index '{index_name}' 中的總資料筆數: {total_rows}")

# 把資料轉換成Pandas DataFrame
pd_df = ed_df.to_pandas()

print(f"成功讀取的筆數: {len(pd_df)}")
print("\n前5筆資料:")
print(pd_df.head())
print("\n資料類型:")
print(pd_df.dtypes)



Index 'random_data_index' 中的總資料筆數: 20000
成功讀取的筆數: 20000

前5筆資料:
          id  random_number random_string
17533  17533            730    3XgoYutGqV
17540  17540            725    5svQfLsnfv
15331  15331            824    JYdBjAGABK
15332  15332             59    P9smoeSk4h
17557  17557             56    2JcanK8yyo

資料類型:
id                int64
random_number     int64
random_string    object
dtype: object


In [9]:
# 從ES中讀取資料
ed_df = eland.DataFrame(es_client=es, es_index_pattern=index_name)

# 詳細的API說明可參考官方文件:
# https://eland.readthedocs.io/en/v8.15.0/reference/supported_apis.html

# 秀出ed_df的物件類型
print(f"ed_df的物件類型: {type(ed_df)}")

# 1. 基本篩選查詢
filtered_df = ed_df[ed_df['random_number'] > 500]
print(f"隨機數大於500的資料數量: {len(filtered_df)}")

# 2. 排序並獲取前10筆記錄（eland不支援pandas的sort_values方法）
sorted_df = ed_df.es_query({
    "sort": [{"random_number": {"order": "desc"}}]
})
pandas_df = ed_df.to_pandas()
sorted_df = pandas_df.sort_values('random_number', ascending=True)
print(f"隨機數最大的10筆記錄:")
print(sorted_df.head(10))

# 3. 分組聚合
grouping_df = ed_df.groupby('random_number').agg(['count'])
grouping_df = grouping_df.sort_values('random_number', ascending=False)
print("\n出現次數最多的5個隨機數:")
print(grouping_df.head(5))


# 4. 複雜條件查詢
complex_query = ed_df[(ed_df['random_number'] > 300) & (ed_df['random_number'] < 700)]
print(f"\n隨機數在300到700之間的資料數量: {len(complex_query)}")

# 5. 文字搜尋
char_query = ed_df['random_string'].to_pandas()
char_query = char_query[char_query.str.contains('a')]
print(f"\n隨機字串中包含'a'的資料數量: {len(char_query)}")



ed_df的物件類型: <class 'eland.dataframe.DataFrame'>
隨機數大於500的資料數量: 10039
隨機數最大的10筆記錄:
          id  random_number random_string
12106  12106              1    9ABGiIe21i
2038    2038              1    ENRb0r7AN6
12871  12871              1    iS8fhY3nXT
15958  15958              1    Kn19wZdLE8
15302  15302              1    9b6VmhDxn6
3029    3029              1    bZSR3D6UKl
16409  16409              1    K2TJaFtKG3
2459    2459              1    NtIMQBud4O
19913  19913              1    XZ7JCHwoEo
8778    8778              1    6It53Zn27t

出現次數最多的5個隨機數:
                 id random_string
              count         count
random_number                    
1000             21            21
999              14            14
998              16            16
997              18            18
996              14            14

隨機數在300到700之間的資料數量: 7959

隨機字串中包含'a'的資料數量: 3002


In [10]:
# 6. 獲取特定ID範圍的資料
id_range_query = ed_df[(ed_df['id'] >= 1000) & (ed_df['id'] < 1010)]
print("\nID從1000到1009的資料:")
print(id_range_query.to_pandas())

# 7. 計算統計資訊
statics = ed_df['random_number'].describe()
print("\n隨機數的統計資訊:")
print(statics)

# 8. 使用ES原生查詢
original_query = ed_df.es_query(
    {
        "query": {
            "range": {
                "random_number": {
                    "gte": 990
                }
            }
        }
    }
)
print(f"\n使用ES原生查詢，隨機數大於等於990的資料數量: {len(original_query)}")


ID從1000到1009的資料:
        id  random_number random_string
1000  1000            264    Btis2dkYKm
1001  1001            307    oSJYtfZ5kO
1002  1002            423    khFm4Fhh5M
1003  1003            916    UZUctHkYJj
1004  1004            518    XbdpvRrrid
1005  1005            229    EoMz4see28
1006  1006            872    3kNJJLIc9Z
1007  1007            361    MOLCUitXnL
1008  1008            780    zdIvHMnXYE
1009  1009            500    FOszDrunrQ

隨機數的統計資訊:
count    20000.000000
mean       499.836250
std        288.577099
min          1.000000
25%        248.560124
50%        502.025191
75%        748.952927
max       1000.000000
Name: random_number, dtype: float64

使用ES原生查詢，隨機數大於等於990的資料數量: 213
